Aby ste mohli spustiť nasledujúce notebooky, ak ste to ešte neurobili, musíte nasadiť model, ktorý používa `text-embedding-ada-002` ako základný model, a nastaviť názov nasadenia v súbore .env ako `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Ďalej načítame Embedding Index do Pandas Dataframe. Embedding Index je uložený v JSON súbore s názvom `embedding_index_3m.json`. Embedding Index obsahuje embeddingy pre každý z prepisov YouTube až do konca októbra 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Ďalej vytvoríme funkciu s názvom `get_videos`, ktorá bude vyhľadávať v Embedding Indexe podľa dotazu. Funkcia vráti 5 najlepších videí, ktoré sú najpodobnejšie dotazu. Funkcia funguje nasledovne:

1. Najprv sa vytvorí kópia Embedding Indexu.
2. Potom sa vypočíta Embedding pre dotaz pomocou OpenAI Embedding API.
3. Následne sa v Embedding Indexe vytvorí nový stĺpec s názvom `similarity`. Stĺpec `similarity` obsahuje kosínovú podobnosť medzi Embeddingom dotazu a Embeddingom pre každý segment videa.
4. Ďalej sa Embedding Index filtruje podľa stĺpca `similarity`. Embedding Index sa filtruje tak, aby obsahoval iba videá, ktoré majú kosínovú podobnosť väčšiu alebo rovnakú ako 0,75.
5. Nakoniec sa Embedding Index zoradí podľa stĺpca `similarity` a vrátia sa 5 najlepších videí.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Táto funkcia je veľmi jednoduchá, len vypisuje výsledky vyhľadávacieho dopytu.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Najprv sa index vkladania načíta do Pandas Dataframe.
2. Potom je používateľ vyzvaný na zadanie dopytu.
3. Následne sa zavolá funkcia `get_videos`, ktorá vyhľadá v indexe vkladania podľa dopytu.
4. Nakoniec sa zavolá funkcia `display_results`, ktorá zobrazí výsledky používateľovi.
5. Používateľ je potom vyzvaný na zadanie ďalšieho dopytu. Tento proces pokračuje, kým používateľ nezadá `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.sk.png)

Budete vyzvaní na zadanie dopytu. Zadajte dopyt a stlačte enter. Aplikácia vráti zoznam videí, ktoré sú relevantné k dopytu. Aplikácia tiež vráti odkaz na miesto vo videu, kde sa nachádza odpoveď na otázku.

Tu je niekoľko dopytov, ktoré môžete vyskúšať:

- Čo je Azure Machine Learning?
- Ako fungujú konvolučné neurónové siete?
- Čo je neurónová sieť?
- Môžem použiť Jupyter Notebooks s Azure Machine Learning?
- Čo je ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zrieknutie sa zodpovednosti**:
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, majte na pamäti, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
